In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip /content/gdrive/Shareddrives/data_face_HUS/data_face_HUS.zip -d "/content/"

Archive:  /content/gdrive/Shareddrives/data_face_HUS/data_face_HUS.zip
replace /content/data_face_HUS/Nguyen_Manh_Trung_20002171/a6.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model
tf.__version__

'2.12.0'

In [7]:
import os
import glob
import cv2
import numpy as np

folder_path = 'data_face_HUS/*/*'

# Find all files in subfolders
files = glob.glob(folder_path, recursive=True)

# Categorize files based on subfolder name
categories = {}

for file in files:
    subfolder = os.path.basename(os.path.dirname(file))
    if subfolder in categories:
        categories[subfolder].append(file)
    else:
        categories[subfolder] = [file]

X_train_label = []
X_train_list = []
X_test_label = []
X_test_list = []
for category, files in categories.items():
    # print(category)
    # for file in files:
    #     print('\t', file)
    train_list = glob.glob('data_face_HUS/'+category+'/*')
    for name in train_list:
        img = cv2.imread(name)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img,(64,64))
            X_train_list.append(img)
            X_train_label.append(category)

In [12]:
model = load_model('/content/gdrive/Shareddrives/data_face_HUS/facenet_keras.h5')

ValueError: ignored

In [8]:
X_train = np.array(X_train_list)
y_train = np.array(X_train_label)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state = 42)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

inp = Input(shape=(28,28,1))
x = Conv2D(filters = 8, kernel_size = 3, activation = 'relu')(inp)
x = MaxPooling2D(pool_size = (2,2))(x)

x = Flatten()(x)
x = Dense(units = 32, activation = 'relu')(x)
x = Dense(units = 2)(x)

cnn = Model(inputs = inp, outputs = x)
img1 = Input(shape = (28,28,1))
img2 = Input(shape = (28,28,1))

f1 = cnn(img1)
f2 = cnn(img2)

d = K.sqrt(K.sum(K.square(f1 - f2),axis = 1, keepdims = True))

model = Model(inputs = [img1,img2], outputs = d)
model.summary()
cnn.summary()

def loss(y_true, y_pred):
  proba = K.exp(-K.square(y_pred))
  return -K.mean(y_true * K.log(proba) + (1-y_true) * K.log(1-proba))

def loss1(y_true, y_pred):
  return K.mean(y_true * K.square(y_pred) + (1-y_true) * K.square(K.maximum(1.0 - y_pred, 0)))

model.compile(optimizer = 'adam', loss = loss1)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def generator(X,y,k = 8):
  unique_labels = np.unique(y)

  while True:
    X1 = []
    X2 = []
    y_batch = []
    for label in unique_labels:
      label_idx = np.where(y == label)[0]
      other_labels = set(unique_labels) - {label}

      for i in range(k):
        i1 = np.random.choice(label_idx)
        i2 = np.random.choice(label_idx)
        # i1 must be different from i2
        while i1 == i2:
          i2 = np.random.choice(label_idx)
        # create positive example
        X1.append(X[i1][:,:,None])
        X2.append(X[i2][:,:,None])
        y_batch.append(1.0)

        # create negative example
        i1 = np.random.choice(label_idx)
        my_label = np.random.choice(list(other_labels))
        i2 = np.random.choice(list(np.where(y == my_label)[0]))
        X1.append(X[i1][:,:,None])
        X2.append(X[i2][:,:,None])
        y_batch.append(0.0)
    yield [np.array(X1) / 255., np.array(X2) / 255.], np.array(y_batch)

# For testing
for pair, y in generator(X_test, y_test):
  print('Batch size: ', len(y))
  idx = np.random.choice(range(len(y)))
  print(pair[0][idx].shape)
  print('Pair label:', y[idx])
  plt.subplot(121)
  plt.imshow(pair[0][idx].reshape(28,28), cmap = 'gray')
  plt.subplot(122)
  plt.imshow(pair[1][idx].reshape(28,28), cmap = 'gray')

  break

In [ ]:
history = model.fit(generator(X_train, y_train, k = 32),
                    steps_per_epoch = 5,
                    epochs = 500,
                    validation_data = generator(X_test, y_test, k = 8),
                    validation_steps = 5)

In [ ]:
plt.plot(history.history['loss'], label = 'Train', c = 'b')
plt.plot(history.history['val_loss'], label = 'Validation', c = 'r')
plt.legend()
plt.xlabel('Epoch', fontsize = 16)
plt.ylabel('Loss', fontsize = 16)

In [ ]:
for pair, y in generator(X_test, y_test):
  y_pred = model.predict(pair)
  print('Batch_size: ', len(y))
  idx = np.random.choice(range(len(y)))
  print('Pair label:', y[idx])
  print('Distance:', y_pred[idx])

  f1 = cnn(pair[0])
  f2 = cnn(pair[1])
  d = np.sqrt(np.sum((f1-f2)**2,axis = 1, keepdims = True))
  print('Distance by features:', d[idx])

  plt.subplot(121)
  plt.imshow(pair[0][idx].reshape(28,28),cmap = 'gray')
  plt.subplot(122)
  plt.imshow(pair[1][idx].reshape(28,28),cmap = 'gray')
  break

In [ ]:
f = cnn.predict(X_test/255.)
p = plt.scatter(f[:,0],f[:,1], c = y_test, s=1)
plt.colorbar(p)

In [ ]:
cnn.save('cnn_loss1.h5')

In [ ]:
from tensorflow.keras.models import load_model
m = load_model('cnn_loss1.h5')

f1 = m.predict(X_test / 255.)
p = plt.scatter(f1[:,0],f1[:,1],c=y_test,s=1)
plt.colorbar(p)

In [ ]:
i = 0
y_true = []
y_pred = []
for pair,y in generator(X_test,y_test):
  f1 = cnn(pair[0])
  f2 = cnn(pair[1])
  d = np.sqrt(np.sum((f1 - f2)**2, axis = 1, keepdims = True))
  y_pred +=list(d.ravel())
  y_true +=list(y)
  i+=1
  if i>500:
    break

In [ ]:
y_pred = np.array(y_pred)
y_true = np.array(y_true)

positive_distances = y_pred[y_true == 1]
negative_distances = y_pred[y_true == 0]

plt.hist(positive_distances, color = 'r', density = True, bins = 20)
plt.hist(negative_distances, color = 'b', density = True, bins = 20)

In [ ]:
thresh = 0.5
y_pred_ = y_pred < thresh
y_pred_.astype('uint8')
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred_))